<a href="https://colab.research.google.com/github/KenjiCpp/Recognize_HW02/blob/main/HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Lấy data
# Data được lưu tại: https://github.com/KenjiCpp/Recognize_HW02

# !git clone https://github.com/KenjiCpp/Recognize_HW02.git

In [ ]:
# Uncomment rồi chạy cell này để xóa ảnh đã xử lý

# import shutil
# shutil.rmtree('Recognize_HW02/processed_images')

In [2]:
# Imports

import pandas as pd
import numpy as np
import tensorflow as tf

import time
import os
import asyncio

import matplotlib.pyplot as plt 
%matplotlib inline

import multiprocessing
from multiprocessing import Pool

import skimage
from skimage import io, transform, color, filters
from skimage.util import img_as_ubyte, img_as_float

import sklearn
from sklearn.decomposition import PCA

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [3]:
# Đọc một file ảnh chưa xử lý

def read_process_save(imageId):
  img      = io.imread(f'Recognize_HW02/images/{imageId}.jpg', as_gray = True)
  smallImg = transform.resize(img, (100, 150)) 
  normImg  = (smallImg - np.min(smallImg)) / (np.max(smallImg) - np.min(smallImg))
  edgeImg  = filters.sobel(normImg)

  io.imsave(f'Recognize_HW02/processed_images/gray/{imageId}.jpg', img_as_ubyte(normImg))
  io.imsave(f'Recognize_HW02/processed_images/edge/{imageId}.jpg', img_as_ubyte(edgeImg))

  return (normImg, edgeImg)

# Đọc một file ảnh đã xử lý

def reread(imageId):
  normImg = img_as_float(io.imread(f'Recognize_HW02/processed_images/gray/{imageId}.jpg', as_gray = True))
  edgeImg = img_as_float(io.imread(f'Recognize_HW02/processed_images/edge/{imageId}.jpg', as_gray = True))

  return (normImg, edgeImg)

# Đọc tập train, test và lưu lại nếu như chưa lưu ảnh đã xử lý

train_df = pd.read_csv('Recognize_HW02/train.csv')
test_df  = pd.read_csv('Recognize_HW02/test.csv')

n_train = train_df.shape[0]
n_test  = test_df .shape[0]

columns = ['pxl_' + str(i) for i in range(150 * 100)]

if not os.path.isdir('Recognize_HW02/processed_images'):
  print('Reading files and processing...')

  os.mkdir('Recognize_HW02/processed_images')
  os.mkdir('Recognize_HW02/processed_images/gray')
  os.mkdir('Recognize_HW02/processed_images/edge')

  with Pool(multiprocessing.cpu_count()) as p:
    train_img = p.map(read_process_save, train_df['image_id'])
    test_img  = p.map(read_process_save, test_df ['image_id'])

else:
  print('Reading processed files...')

  with Pool(multiprocessing.cpu_count()) as p:
    train_img = p.map(reread, train_df['image_id'])
    test_img  = p.map(reread, test_df ['image_id'])

print('Done!')

train_X_gray = pd.DataFrame.from_records(np.array([img[0] for img in train_img]).reshape(n_train, 150 * 100), columns = columns)
train_X_edge = pd.DataFrame.from_records(np.array([img[1] for img in train_img]).reshape(n_train, 150 * 100), columns = columns)
test_X_gray  = pd.DataFrame.from_records(np.array([img[0] for img in test_img ]).reshape(n_test , 150 * 100), columns = columns)
test_X_edge  = pd.DataFrame.from_records(np.array([img[1] for img in test_img ]).reshape(n_test , 150 * 100), columns = columns)

train_y = train_df[['healthy', 'multiple_diseases', 'rust', 'scab']]

Reading files and processing...
Done!


In [4]:
pca = PCA()

train_X = pca.fit_transform(train_X_gray)

model = Sequential()
model.add(Dense(800, activation = 'linear', input_dim = 1821))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(4  , activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
model.fit(train_X, train_y)

57/57 [==============================] - 1s 13ms/step - loss: 1.5639 - accuracy: 0.3591


In [12]:
A = train_y.idxmax(axis=1)
B = pd.DataFrame.from_records(model.predict(train_X), columns = train_y.columns).idxmax(axis=1)

sum([A[i] == B[i] for i in range(1821)])

1627